# 任务一 ： 关联规则挖掘
## 1.1 数据离散化

In [3]:
"""我们将连续变量如温度和湿度等进行离散化。可以使用pd.cut方法将连续值分成多个类别区间"""
import pandas as pd

# 读取数据
data = pd.read_csv('fyx.csv')

# 温度离散化
data['temperature_discrete'] = pd.cut(data['Temperature (C)'], bins=[-10, 0, 10, 20, 30], labels=["非常低温", "低温", "中温", "高温"])

# 湿度离散化
data['humidity_discrete'] = pd.cut(data['Humidity'], bins=[0, 0.5, 0.75, 1], labels=["低湿度", "中湿度", "高湿度"])

# 风速离散化
data['windspeed_discrete'] = pd.cut(data['Wind Speed (km/h)'], bins=[0, 10, 20, 30], labels=["低速风", "中速风", "高速风"])

# 查看离散化结果
print(data[['Temperature (C)', 'temperature_discrete', 'Humidity', 'humidity_discrete', 'Wind Speed (km/h)', 'windspeed_discrete']].head())
data.to_csv('离散化数据.csv', index=False)
print(data.dtypes)


   Temperature (C) temperature_discrete  Humidity humidity_discrete  \
0         0.577778                   低温      0.89               高湿度   
1         1.161111                   低温      0.85               高湿度   
2         1.666667                   低温      0.82               高湿度   
3         1.711111                   低温      0.82               高湿度   
4         1.183333                   低温      0.86               高湿度   

   Wind Speed (km/h) windspeed_discrete  
0            17.1143                中速风  
1            16.6152                中速风  
2            20.2538                高速风  
3            14.4900                中速风  
4            13.9426                中速风  
Formatted Date                object
Summary                       object
Precip Type                   object
Temperature (C)              float64
Apparent Temperature (C)     float64
Humidity                     float64
Wind Speed (km/h)            float64
Wind Bearing (degrees)       float64
Visibility (km)          

## 1.2 构建事务集
将每条记录转换为事务，包含离散化后的特征值。

In [13]:
# 选择相关的离散化特征
transactions = data[['temperature_discrete', 'humidity_discrete', 'windspeed_discrete', 'Summary', 'WindDirection','Season','TimeOfDay','Daily Summary']].astype(str)

# 将每行记录转换为事务
transactions_list = transactions.values.tolist()
# 打印前5条记录
for i in range(5):
    print(transactions_list[i])

['低温', '高湿度', '中速风', 'Partly Cloudy', '东南', '冬季', '凌晨', 'Mostly cloudy throughout the day.']
['低温', '高湿度', '中速风', 'Mostly Cloudy', '东南', '冬季', '凌晨', 'Mostly cloudy throughout the day.']
['低温', '高湿度', '高速风', 'Mostly Cloudy', '东南', '冬季', '凌晨', 'Mostly cloudy throughout the day.']
['低温', '高湿度', '中速风', 'Overcast', '东南', '冬季', '凌晨', 'Mostly cloudy throughout the day.']
['低温', '高湿度', '中速风', 'Mostly Cloudy', '东', '冬季', '凌晨', 'Mostly cloudy throughout the day.']


## 1.3 使用Apriori算法进行关联规则挖掘
使用mlxtend库中的Apriori算法挖掘频繁项集。
各列的解释
* antecedents（前件）：规则的左侧部分，表示条件。
* consequents（后件）：规则的右侧部分，表示结果。
* antecedent support（前件支持度）：在数据集中，前件出现的频率。
* consequent support（后件支持度）：后件出现的频率。
* support（支持度）：同时满足前件和后件的记录频率。
* confidence（置信度）：给定前件的情况下，后件发生的概率。计算公式为 support(antecedents ∩ consequents) / support(antecedents)。
* lift（提升度）：后件在给定前件情况下发生的可能性相对于其独立发生的可能性。计算公式为 confidence / support(consequents)。值大于1表示前件对后件有正向影响。
* leverage（杠杆值）：反映前件和后件共同发生的超出随机发生的程度。计算公式为 support(antecedents ∩ consequents) - (support(antecedents) * support(consequents))。值越大表示相关性越强。
* conviction：衡量前件发生时后件不发生的可能性。计算公式为 (1 - support(consequents)) / (1 - confidence)。值越大表示前件对后件的影响越强。
* zhangs_metric：Zhang的指标，用于评估规则的重要性，通常在某些特定的研究中使用。

In [15]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# 转换为适合Apriori算法的格式
te = TransactionEncoder() # 定义模型
transactions_encoded = te.fit(transactions_list).transform(transactions_list) # 转换数据
df_transactions = pd.DataFrame(transactions_encoded, columns=te.columns_) # 转换为DataFrame

# 设定最小支持度和置信度阈值并挖掘频繁项集
frequent_itemsets = apriori(df_transactions, min_support=0.05, use_colnames=True)

# 生成关联规则
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.6)


In [19]:
filtered_rules = rules[
    (rules['support'] >= 0.05) & 
    (rules['confidence'] >= 0.7) & 
    (rules['lift'] >= 1.5)
]

# 输出关联规则
print(filtered_rules)
filtered_rules.to_csv('关联规则.csv', index=False)


        antecedents consequents  antecedent support  consequent support  \
2           (Foggy)       (高湿度)            0.074127            0.552344   
9            (非常低温)        (冬季)            0.105964            0.247146   
11             (凌晨)       (高湿度)            0.249987            0.552344   
14           (非常低温)       (高湿度)            0.105964            0.552344   
15     (Foggy, 低速风)       (高湿度)            0.051986            0.552344   
25   (低温, Overcast)       (高湿度)            0.083274            0.552344   
26  (低速风, Overcast)       (高湿度)            0.068185            0.552344   
27   (冬季, Overcast)       (高湿度)            0.069564            0.552344   
34         (下午, 高温)       (低湿度)            0.076274            0.161186   
43         (凌晨, 夏季)        (中温)            0.062948            0.345156   
46        (高湿度, 夏季)        (中温)            0.096724            0.345156   
47        (中湿度, 高温)        (夏季)            0.075911            0.251843   
53        (低速风, 低温)      

在上述数据中，我们看到一系列的关联规则，这些规则描述了不同气象条件（如温度、湿度、风速等）之间的关系。通过分析这些规则的度量值，我们可以评估它们的有用性和可靠性。

### 关联规则的基本概念

- **Antecedent（前件）**：规则的左侧，即如果部分。
- **Consequent（后件）**：规则的右侧，即则部分。
- **Support（支持度）**：同时包含前件和后件的数据项集出现的频率。
- **Confidence（置信度）**：给定前件情况下，后件出现的概率。
- **Lift（提升度）**：衡量规则的有效性，表示前件与后件之间关联性的强度。
- **Leverage（杠杆率）**：表示前件和后件一起发生的概率与期望概率的差异。
- **Conviction（确信度）**：衡量当规则为假时的意外程度。
- **Zhang's Metric（张氏度量）**：综合考虑了规则的准确性和覆盖率。

### 规则分析

1. **规则："Foggy" -> "高湿度"**
   - **Confidence (置信度)**: 0.9959，非常高，意味着几乎可以确定当天气“雾”时，湿度会很高。
   - **Lift (提升度)**: 1.8031，表明“雾”和“高湿度”之间存在较强的正相关关系。
   - **有用性**: 对于农业、交通等行业来说，这个规则非常有用，可以帮助预测恶劣天气，采取预防措施。

2. **规则："非常低温" -> "冬季"**
   - **Confidence (置信度)**: 0.8354，较高，说明当气温非常低时，很可能是冬季。
   - **Lift (提升度)**: 3.3801，表示“非常低温”与“冬季”之间有很强的联系。
   - **有用性**: 对于季节性活动规划或能源需求预测等领域，此规则具有实际应用价值。

3. **规则："凌晨" -> "高湿度"**
   - **Confidence (置信度)**: 0.8405，高，表示凌晨时分湿度较高的可能性很大。
   - **Lift (提升度)**: 1.5216，显示“凌晨”与“高湿度”之间有一定的相关性。
   - **有用性**: 对于气象预报或户外活动安排，这条规则可以提供参考信息。

4. **规则："非常低温" & "高湿度" -> "冬季"**
   - **Confidence (置信度)**: 0.8404，较高，意味着当同时出现“非常低温”和“高湿度”时，几乎可以断定是冬季。
   - **Lift (提升度)**: 3.4003，非常高的提升度反映了这两者组合对冬季的强烈指示作用。
   - **有用性**: 这个规则对于季节性气候变化的研究特别有价值。

### 总结

从以上分析可以看出，这些关联规则提供了关于特定气象条件下其他气象条件发生的概率的信息。对于需要根据天气状况做出决策的领域，比如农业灌溉、交通管理、旅游规划等，这些规则都是非常有用的工具。然而，需要注意的是，虽然这些规则在统计上有效，但在实际应用时还需要结合其他因素进行综合判断。例如，地理位置、季节变化等因素都可能影响规则的应用效果。因此，在利用这些规则时，应该考虑到这些外部因素的影响。

# 任务二 ： 异常检测
## 2.1 特征选择与数据标准化处理
选择相关的特征，如温度、湿度、风速，并进行标准化处理。

In [20]:
from sklearn.preprocessing import StandardScaler

# 选择用于检测异常的特征
features = data[['Temperature (C)', 'Humidity', 'Wind Speed (km/h)','Apparent Temperature (C)','Visibility (km)','Pressure (millibars)']]

# 标准化处理
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)


## 2.2 使用孤立森林算法进行异常检测

In [36]:
from sklearn.ensemble import IsolationForest

# 初始化数据
remaining_data = data.copy()

# 逐个维度进行异常检测
for feature in features.columns:
    # 训练孤立森林模型，仅使用当前特征
    feature_values = remaining_data[feature].values.reshape(-1, 1)
    isf_feature = IsolationForest(contamination=0.05, random_state=42)
    
    # 拟合模型并预测异常
    feature_anomaly_score = isf_feature.fit_predict(feature_values)

    # 标记异常
    remaining_data[f'{feature}_anomaly'] = feature_anomaly_score < 0  # 小于0表示异常
    
    # 打印异常原因并删除异常数据,只打印异常数据
    anomalies = remaining_data[remaining_data[f'{feature}_anomaly']]
    anomalies.to_csv(f'{feature}_异常数据.csv'.replace(' ','_').replace('/','_'), index=False)
    if not anomalies.empty: 
        print(f"{feature} 维度异常数据：")
        print(anomalies)
        remaining_data = remaining_data[~remaining_data[f'{feature}_anomaly']]  # 删除异常数据
    
    # 删除标记列
    remaining_data.drop(columns=[f'{feature}_anomaly'], inplace=True)

# 最终更新的数据
print("处理后的数据：")
print(remaining_data)
remaining_data.to_csv('异常数据处理后.csv', index=False)


Temperature (C) 维度异常数据：
                  Formatted Date        Summary Precip Type  Temperature (C)  \
196    2006-01-09 04:00:00+00:00          Clear        snow        -5.094444   
197    2006-01-09 05:00:00+00:00          Clear        snow        -5.138889   
198    2006-01-09 06:00:00+00:00          Clear        snow        -6.205556   
199    2006-01-09 07:00:00+00:00  Partly Cloudy        snow        -6.044444   
200    2006-01-09 08:00:00+00:00  Partly Cloudy        snow        -5.138889   
...                          ...            ...         ...              ...   
93756  2016-09-11 15:00:00+00:00  Mostly Cloudy        rain        32.000000   
93757  2016-09-11 16:00:00+00:00  Mostly Cloudy        rain        30.427778   
93777  2016-09-12 12:00:00+00:00  Partly Cloudy        rain        31.038889   
93778  2016-09-12 13:00:00+00:00  Partly Cloudy        rain        31.766667   
93779  2016-09-12 14:00:00+00:00  Partly Cloudy        rain        30.711111   

       Apparent